In [1]:
import emails
import hme
import dupl
import uts
import re
import csv

In [5]:
emailstext = emails.getAll("allfilenames.txt")

In [6]:
def machine(bmpgl_emailstext, METHOD, R, ):
    Sdeqs = []
    if METHOD == 'bm':
        listbms = re.split("BOOKMARKend---",bmpgl_emailstext)
        for bm in listbms:
            listTS = uts.getUnixTS(bm) #1*
            listTSr = uts.getUnixTSr(bm) #1*
            listSo = dupl.findRawSenders_bm(bm) #2*
            deqfile = re.findall("deq.*.txt",bm) #0
            if deqfile == [] or listTS == [] or listSo == []:
                pass
            else:
                dfroot = re.findall("(deq.._b[0-9]*_[0-9]+_[0-9]+_)[0-9]+.txt",bm)
                dfroot_X = str(dfroot[0] + "X")
                if R == "round":
                    Sdeq = (dfroot_X,listTSr,listSo)
                    Sdeqs.append(Sdeq)
                elif R == "no round":
                    Sdeq = (dfroot_X,listTS,listSo)
                    Sdeqs.append(Sdeq)
                else:
                    print("""do you want to round unix ts? print 'round' or 'no round' in third pos""")
                    break

        return Sdeqs
    elif METHOD == 'pg':
        Sdeqs = []
        listpgs = re.split("---endPAGE",bmpgl_emailstext)
        for pg in listpgs:
            listTS = uts.getUnixTS(pg) #1*
            listTSr = uts.getUnixTSr(pg) #1*
            listSo = dupl.findRawSenders_pg(pg) #2*
            deqfile = re.findall("deq.*.txt",pg) #0
            if deqfile == [] or listTS == [] or listSo == []:
                pass
            else:
                if R == "round":
                    Sdeq = (deqfile[0],listTSr,listSo)#listRo,listSub)
                    Sdeqs.append(Sdeq)
                elif R == "no round":
                    Sdeq = (deqfile[0],listTS,listSo)#listRo,listSub)
                    Sdeqs.append(Sdeq)
                else:
                    print("""do you want to round unix ts? print 'round' or 'no round' in third pos""")
                    break
        return Sdeqs
    else:
        print("invalid METHOD. try 'bm' or 'pg' in second position")

In [7]:
#STEP 1:
#format -- ('deqfile',[list of uts],[list of senders])
deqbm = machine(emailstext,'bm','round')
deqpg = machine(emailstext,'pg','round')

/home/bitnami/.local/lib/python3.5/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/bitnami/.local/lib/python3.5/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/bitnami/.local/lib/python3.5/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDY identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/bitnami/.local/lib/python3.5/site-packages/dat

In [122]:
deqbm[0:5]

[('deq01_b33_3579_3579_X', [1412080600], ['Busch, Stephen']),
 ('deq01_b378_4225_4226_X',
  ['NOTIMESTAMP', 1437739700],
  ['NOSENDER', 'Wurfel, Brad']),
 ('deq01_b150_3778_3782_X',
  ['NOTIMESTAMP', 1426777500, 1426767300, 1426261600, 1426012800],
  ['NOSENDER',
   'Busch, Stephen',
   'Henry, James',
   'Busch, Stephen',
   'Henry, James']),
 ('deq01_b412_4298_4299_X',
  [1380098700, 1380096100, 1380033800],
  ['Busz, Rick', 'Cook, Pat', 'Rusz, Rick']),
 ('deq01_b355_4185_4186_X',
  [1402579200, 1402579000],
  ['Wurfel, Brad', 'Busch, Stephen'])]

In [33]:
#STEP 3a:
#takes the long (>10)
sdeqbm = dupl.shortbm(deqbm,11) #takes <11
ldeqpg = dupl.longpg(deqpg,10) #takes >10
total = len(sdeqbm) + len(ldeqpg)
total

10799

In [9]:
#STEP 3b: take only the files where len(utslist) = len(senderlist)
sdeqslist = []
for deqs in sdeqbm:
    if len(deqs[1]) == len(deqs[2]):
        sdeqslist.append(deqs)
(sdeqslist)

ldeqslist = []
for deqs in ldeqpg:
    if len(deqs[1]) == len(deqs[2]):
        ldeqslist.append(deqs)
lpg_sbm = list(ldeqslist+sdeqslist)
len(ldeqslist) + len(sdeqslist)

10249

In [35]:
lpg_sbm[30:35]

[('deq01_b1_1_1275_110.txt',
  [1431597000, 1431593600, 1431529600],
  ['Rosenthal, Adam', 'Glasgow, Michael', 'Rosenthal, Adam']),
 ('deq01_b1_1_1275_111.txt',
  [1431518000, 1431448300],
  ['Glasgow, Michael', 'Rosenthal, Adam']),
 ('deq01_b1_1_1275_113.txt',
  [1431529700, 1431518000, 1431448300],
  ['Rosenthal, Adam', 'Glasgow, Michael', 'Rosenthal, Adam']),
 ('deq01_b1_1_1275_115.txt',
  [1423492600, 1423490400, 1423488000],
  ['Rosenthal, Adam', 'Glasgow, Michael', 'Rosenthal, Adam']),
 ('deq01_b1_1_1275_118.txt', [1444924500], ['Feuerstein, Heather'])]

In [ ]:
# TYPE A DUPS 

In [125]:
#STEP 4A: change unit of analysis to 'email'
em_deqf = dupl.resortbyUTS(lpg_sbm)
em_deqf[535:545]

[((1342007200, 'Lee, Michelle'),
  ['deq01_b376_4222_4223_X', 'deq12_b208_874_874_X']),
 ((1342013200, 'Benzie, Richard'), ['deq12_b208_874_874_X']),
 ((1343635700, 'Deltoral, Miguel'), ['deq21_b1_1_1811_1285.txt']),
 ((1343635900, 'Deltoral, Miguel'), ['deq21_b1_1_1811_1285.txt']),
 ((1345481800, 'Redding, Jim'), ['deq11_b183_975_983_X']),
 ((1345482900, 'Ashford, Stephen'), ['deq11_b183_975_983_X']),
 ((1345483000, 'Redding, Jim'), ['deq11_b183_975_983_X']),
 ((1345544900, 'Ashford, Stephen'), ['deq11_b183_975_983_X']),
 ((1345559700, 'Redding, Jim'), ['deq11_b183_975_983_X']),
 ((1346164900, 'Rosenthal, Adam'), ['deq11_b52_127_127_X'])]

In [127]:
em_deqf1 = [] #excluding NOTIMESTAMP
for elem in em_deqf:
    us = elem[0]
    deqf = elem[1]
    uts=us[0]
    sndr=us[1]
    if uts != 'NOTIMESTAMP':
        em_deqf1.append(elem)

In [128]:
# generating frequency data for Type A #

In [130]:
frequ = []
for ls in em_deqf1:
    us = ls[0]
    deq = ls[1]
    uts = us[0]
    frequ_item=(uts,int(len(deq)))
    frequ.append(frequ_item)
frequ.sort(key=dupl.takeSecond)
#frequ #list of (uts,freq of uts)

fof = []
for item in frequ:
    freq = item[1]
    fof.append(freq)
j = hme.freqs(fof)
#list of (freq of uts, freq of freq of uts)

In [14]:
# DATA FOR FIGs 1 & 2: freq of type A dups
with open('unixfreqfreq.csv', mode='w') as usd_file:
        table = csv.writer(usd_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for elem in j:
            freq = elem[0]
            freqoffreq = elem[1]
            table.writerow([freq,freqoffreq])

In [15]:
# TYPE B DUPS

In [158]:
# STEP 4B: rearrange to ((emails in file),[deqfiles with that email sequence])
fdic1 = {}
ndl = {}
news = []
for things in em_deqf:
    deqflist = things[1]
    us = things[0]
    usl = list(us)
    for deqs in deqflist:
        new = (deqs,us)
        news.append(new)
        deqf = new[0]
        us0 = new[1]
        newdic = {}
        if deqf not in [*ndl]:
            newdic[deqf] = [us0]
            ndl.update(newdic)
        else:
            ndl[deqf].append(us0)
            pass
predeqf_em = dupl.tolist(ndl)

In [166]:
def takeX(elem):
    for elems in elem[0]:
        if elems[0] == 'NOTIMESTAMP':
            return 0
        else:
            return (elems[0])

In [190]:
roo = []
ndl={}
for des in predeqf_em:
    emails = des[1]
    deq = des[0]
    ro = (emails,deq)
    roo.append(ro)
roo.sort(key=dupl.takeSecond)
for roos in roo:
    newdic = {}
    email = tuple(roos[0])
    deqf = roos[1]
    if email not in [*ndl]:
        newdic[email] = [deqf]
        ndl.update(newdic)
    else:
        ndl[email].append(deqf)
        pass
ems_deqfB = dupl.tolist(ndl)
ems_deqfB.sort(key=takeX)
(ems_deqfB[500:505])

[(((1383839500, 'Jansen, David'),
   (1383901000, "O'Malia, John"),
   (1383901100, "O'Malia, John"),
   (1384243300, 'Prysby, Mike')),
  ['deq04_b577_6019_6376_275.txt']),
 (((1383899200, 'Busch, Stephen'),), ['deq11_b450_1943_1943_X']),
 (((1383901100, "O'Malia, John"),
   (1384243300, 'Prysby, Mike'),
   (1384252400, 'Busch, Stephen')),
  ['deq03_b1_1_4717_4241.txt']),
 (((1385118600, 'Glasgow, Michael'), (1385124400, 'Forbes, Kevin')),
  ['deq11_b3_6_6_X']),
 (((1385400600, 'Wurfel, Brad'),
   (1385453400, 'Hare, Chris'),
   (1385454800, 'Bloemker, Jon'),
   (1385462800, 'Bloemker, Jon'),
   (1385463500, 'Benzie, Richard')),
  ['deq21_b1_1_1811_806.txt'])]

In [184]:
newls = []
for ls in ems_deqfB:
    new = (ls[0],len(ls[1]))
    newls.append(new)
newls.sort(key=dupl.takeSecond)

In [187]:
roo1 = []
edB_freq = []
for roos in newls:
    roo1.append(roos[1])
edB_freq = hme.freqs(roo1)
edB_freq.sort(key=dupl.takeFirst)

In [188]:
# DATA FOR FIGs 3 & 4: freq of type B dups
with open('typeB_dups1.csv', mode='w') as usd_file:
        table = csv.writer(usd_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for elem in hroo:
            email = elem[0]
            deqfcount = elem[1]
            table.writerow([email,deqfcount])

In [114]:
deqs = []
for things in em_deqf: #((uts, sender),[deqflist])
    deqlist = things[1]
    if len(deqlist) > 1:
        deqs.extend(deqlist)
deqsd = list(dict.fromkeys(deqs))
len(deqsd) # num of files with type A dups in them, combo of bookmarks and pgs

8517

In [ ]:
# END #

In [115]:
dd1 = []
for dd in deqsd:
    dfroot = re.findall("(deq.._b[0-9]*_[0-9]+_[0-9]+_)[0-9]+.txt",dd)
    if dfroot != []:
        dfroot_X = str(dfroot[0] + "X")
        dd1.append(dfroot_X)
    else:
        dd1.append(dd)
dd1d = list(dict.fromkeys(dd1))
len(dd1d) # deqsd, but only bookmarks

2499

In [116]:
2499/5048

0.49504754358161646

In [104]:
f = open("allfilenames.txt", "r+")
deqsstr = f.read()
deqfn = re.findall("deq0[1234].*|deq1[012456789].*|deq2[0145].*",deqsstr)

bookmarks_pg = []
for deqs in deqfn:
    dfroot = (re.findall("(deq.._b[0-9]*_[0-9]+_[0-9]+_)[0-9]+.txt",deqs))
    if dfroot != []:
        dfroot_X = str((dfroot[0]) + "X")
        bookmarks_pg.append(dfroot_X)
    else:
        pass
bookmarks = list(dict.fromkeys(bookmarks_pg))
print(str(len(bookmarks_pg)) + " pages")
print(str(len(bookmarks)) + " bookmarks")

51768 pages
5048 bookmarks


In [107]:
import statistics
pgz = []
pgz1 = []
for root in bookmarks:
    m1 = re.findall("deq[0-9][0-9]_b[0-9]+_([0-9]+)_[0-9]+_X",root)
    m2 = re.findall("deq[0-9][0-9]_b[0-9]+_[0-9]+_([0-9]+)_X",root)
    pgs = int(m2[0]) - int(m1[0])
    pgz.append(pgs)
for pg in pgz:
    new = int(pg + 1)
    pgz1.append(new)
    pgz1.sort()
print("mean: " + str(statistics.mean(pgz1)))
print("median: " + str(statistics.median(pgz1)))
print("stdev: " + str(statistics.stdev(pgz1)))
print("range: " + str(pgz1[0]) + " - " + str(pgz1[-1]))

mean: 10.331418383518225
median: 2.0
stdev: 120.6362767228428
range: 1 - 4717
